<a href="https://colab.research.google.com/github/mehdiiiii786/finance/blob/main/Daily_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 9.8 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models, EfficientFrontier, objective_functions
import warnings

# ==========================================
# 0. SETTINGS (DEBUG MODE)
# ==========================================
TOTAL_CAPITAL = 100000
SKIP_LIVE_CHECK = True   # <--- SET TO TRUE ON WEEKENDS (Fixes the Sunday Glitch)
GAP_THRESHOLD = -0.015
warnings.filterwarnings("ignore")

print(f"🚀 PROTOCOL OMEGA (DEBUG MODE): INITIALIZING...")

# ==========================================
# 1. THE MEGA-UNIVERSE
# ==========================================
sector_map = {
    '💻 TECH (SEMI)':  ['NVDA', 'AMD', 'AVGO', 'MU', 'INTC', 'TSM', 'QCOM', 'ARM', 'SMCI'],
    '📱 TECH (BIG)':   ['AAPL', 'MSFT', 'GOOGL', 'META', 'ORCL', 'CSCO', 'IBM'],
    '☁️ SOFTWARE':     ['PLTR', 'CRM', 'MSTR', 'NET', 'CRWD', 'SNOW', 'DDOG', 'APP', 'ADBE'],
    '🏦 FINANCE':      ['JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'C', 'AXP', 'V', 'MA'],
    '🪙 FINTECH':      ['COIN', 'HOOD', 'PYPL', 'SQ', 'SOFI', 'MARA', 'RIOT', 'CLSK'],
    '🛍️ CONSUMER':     ['TSLA', 'AMZN', 'HD', 'MCD', 'NKE', 'SBUX', 'LULU', 'CMG', 'DKNG'],
    '🛒 STAPLES':      ['WMT', 'COST', 'KO', 'PEP', 'PG', 'TGT', 'CL', 'MO'],
    '⚡ ENERGY':       ['XOM', 'CVX', 'COP', 'EOG', 'SLB', 'VLO', 'OXY', 'HAL', 'MPC'],
    '💊 HEALTH':       ['LLY', 'UNH', 'JNJ', 'PFE', 'MRK', 'ABBV', 'VRTX', 'ISRG', 'NVO'],
    '🏭 INDUSTRIAL':   ['GE', 'CAT', 'DE', 'HON', 'UPS', 'BA', 'LMT', 'RTX', 'UBER'],
    '⛏️ MATERIALS':    ['FCX', 'NEM', 'SCCO', 'AA', 'CLF', 'LIN', 'SHW', 'DOW']
}

all_tickers = list(set([t for s in sector_map.values() for t in s] + ['SPY']))
print(f"📋 Watchlist size: {len(all_tickers)} tickers.")

# ==========================================
# 2. DATA ENGINE
# ==========================================
print(f"📡 Downloading 6 months of data...")
try:
    data = yf.download(all_tickers, period="6mo", interval="1d", progress=False, auto_adjust=True)
except Exception as e:
    print(f"❌ CRITICAL DOWNLOAD ERROR: {e}")
    exit()

if isinstance(data.columns, pd.MultiIndex):
    close = data['Close'] if 'Close' in data.columns else data
else:
    close = data['Close'] if 'Close' in data else data

# DEBUG CHECK 1: DID WE GET DATA?
if close.empty:
    print("❌ ERROR: Download returned empty dataframe. Check your internet connection.")
    exit()
else:
    print(f"✅ Data received. Rows: {len(close)}")

if 'SPY' not in close.columns:
    print("❌ ERROR: 'SPY' (Benchmark) is missing from download. Cannot proceed.")
    exit()

market_prices = close['SPY']

# ==========================================
# 3. FILTERING (BETA & MOMENTUM)
# ==========================================
print("⚙️ calculating metrics...")

metrics = []
for sector, tickers in sector_map.items():
    for t in tickers:
        if t not in close.columns: continue

        # Robust cleanup
        pair_df = pd.DataFrame({'Stock': close[t], 'Market': market_prices}).dropna()
        if len(pair_df) < 30: continue

        stock_ret = pair_df['Stock'].pct_change().dropna()
        market_ret = pair_df['Market'].pct_change().dropna()

        # Beta Calc
        if len(stock_ret) < 30: continue
        cov = np.cov(stock_ret, market_ret)[0][1]
        var = np.var(market_ret)
        beta = cov / var

        # Momentum Calc
        price_now = pair_df['Stock'].iloc[-1]
        price_then = pair_df['Stock'].iloc[-20]
        momentum = (price_now / price_then) - 1

        metrics.append({
            'Ticker': t, 'Sector': sector, 'Beta': beta,
            'Momentum': momentum, 'Price': price_now
        })

df_metrics = pd.DataFrame(metrics).set_index('Ticker')

# DEBUG CHECK 2: HOW MANY PASSED?
print(f"📊 metrics calculated for {len(df_metrics)} stocks.")

# RELAXED FILTERS (In case market is bad)
# Beta > 0.5 (was 0.85) | Momentum > -0.05 (Allow slightly red stocks)
survivors = df_metrics[
    (df_metrics['Beta'] > 0.5) &
    (df_metrics['Momentum'] > -0.05)
].sort_values(by='Momentum', ascending=False)

preliminary_candidates = survivors.head(15).index.tolist()
print(f"⚔️ Survivors after filter: {len(preliminary_candidates)}")

if len(preliminary_candidates) == 0:
    print("❌ FILTER TOO STRICT: No stocks passed Beta/Momentum check.")
    print("   (Try lowering Beta threshold or allowing negative momentum)")
    exit()

# ==========================================
# 4. MORNING SAFETY CHECK
# ==========================================
final_candidates = []

if SKIP_LIVE_CHECK:
    print("⚠️ SKIPPING LIVE GAP CHECK (Weekend Mode). Using all candidates.")
    final_candidates = preliminary_candidates
else:
    print(f"🔍 Running Live Gap Check...")
    for t in preliminary_candidates:
        try:
            fast_info = yf.Ticker(t).fast_info
            last = fast_info['last_price']
            prev = fast_info['previous_close']

            # Handle "None" on weekends
            if last is None or prev is None:
                print(f"   ⚠️ No live data for {t} (Weekend?), keeping it.")
                final_candidates.append(t)
                continue

            gap = (last - prev) / prev

            if gap < GAP_THRESHOLD:
                print(f"   ❌ BLOCKED: {t} (Gap: {gap*100:.2f}%)")
            else:
                final_candidates.append(t)
        except:
            final_candidates.append(t)

print(f"💎 Final Candidate Count: {len(final_candidates)}")

# ==========================================
# 5. OPTIMIZATION
# ==========================================
if len(final_candidates) > 0:
    print("\n🧮 Optimizing Portfolio...")

    subset_prices = close[final_candidates].dropna()
    mu = expected_returns.ema_historical_return(subset_prices, span=15, frequency=252)
    S = risk_models.sample_cov(subset_prices)

    # Diversified: Max 20% per stock
    ef = EfficientFrontier(mu, S, weight_bounds=(0.0, 0.20))
    ef.max_quadratic_utility(risk_aversion=0.5)
    cleaned_weights = ef.clean_weights()

    print("\n" + "="*85)
    print(f"🏆 OMEGA PORTFOLIO (DEBUGGED) | CAPITAL: ${TOTAL_CAPITAL:,}")
    print("="*85)
    print(f"{'TICKER':<6} | {'SECTOR':<18} | {'ALLOC':<8} | {'SHARES':<8} | {'BETA':<5}")
    print("-" * 85)

    for ticker, weight in pd.Series(cleaned_weights).sort_values(ascending=False).items():
        if weight > 0.01:
            sec = df_metrics.loc[ticker, 'Sector']
            beta_val = df_metrics.loc[ticker, 'Beta']
            price = df_metrics.loc[ticker, 'Price']

            dollar_amt = weight * TOTAL_CAPITAL
            shares = int(dollar_amt / price)

            print(f"{ticker:<6} | {sec:<18} | {weight*100:.1f}%   | {shares:<8} | {beta_val:.2f}")

    print("-" * 85)
else:
    print("❌ ERROR: No candidates left for optimization.")

🚀 PROTOCOL OMEGA (DEBUG MODE): INITIALIZING...
📋 Watchlist size: 96 tickers.
📡 Downloading 6 months of data...


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SQ']: YFPricesMissingError('possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")')


✅ Data received. Rows: 127
⚙️ calculating metrics...
📊 metrics calculated for 94 stocks.
⚔️ Survivors after filter: 15
⚠️ SKIPPING LIVE GAP CHECK (Weekend Mode). Using all candidates.
💎 Final Candidate Count: 15

🧮 Optimizing Portfolio...

🏆 OMEGA PORTFOLIO (DEBUGGED) | CAPITAL: $100,000
TICKER | SECTOR             | ALLOC    | SHARES   | BETA 
-------------------------------------------------------------------------------------
AA     | ⛏️ MATERIALS       | 20.0%   | 456      | 2.44
LULU   | 🛍️ CONSUMER        | 20.0%   | 105      | 1.89
APP    | ☁️ SOFTWARE        | 20.0%   | 28       | 2.12
INTC   | 💻 TECH (SEMI)      | 20.0%   | 482      | 2.08
CRM    | ☁️ SOFTWARE        | 20.0%   | 76       | 0.97
-------------------------------------------------------------------------------------
